# Домашнее задание №1
Добрынина Анастасия, БКЛ-213

In [1]:
from tqdm import tqdm
import pymorphy2
from nltk import word_tokenize
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from sklearn.metrics import accuracy_score
from collections import Counter

C:\Users\79998\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
morph = pymorphy2.MorphAnalyzer()

## 1. Парсинг
*Сначала нам надо скачать данные -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления "тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. (2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга)*

Буду анализировать отзывы на онлайн кинотеатры с сайта [отзовик](https://otzovik.com/?&capt4a=4321694895366182). На выбранном сайте есть отзывы на 7 разных кинотеатров, возьмем поровну отзывов на каждый из них. Так как киноеатров немного, ссылки с отзывами на каждый будет удобней собрать руками.

Один кинотеатр лучше оставить для теста. Так я точно могу быть уверенной, что определение тональности не переобучено на отзывы на конкретные кинотеатры, а может правильно определять тональность отзыва на любой киннотеатр

In [3]:
cinemas = ['https://otzovik.com/reviews/interaktivnoe_televidenie_wink_russia/',
           'https://otzovik.com/reviews/ivi_ru-besplatniy_videoservis_s_licenzionnim_polnometrazhnim_kontentom/',
           'https://otzovik.com/reviews/more_tv-onlayn_kinoteatr/',
          'https://otzovik.com/reviews/tv-kanal_netflix/',
          'https://otzovik.com/reviews/amediateka_ru-onlayn_kinoteatr/',
           'https://otzovik.com/reviews/kinopoisk_ru-onlayn-kinoteatr/']
cinemas_test = 'https://otzovik.com/reviews/okko_tv-onlayn_kinoteatr/'

Напишем функцию, которая будет открывать отзывы и собирать нужный текст.
- к текстам хороших отзывов будет относится основное содержнаие 5-звездочных отзвов
- к текстам плохих отзывов соответсвенно будет относится основное содержание 1-звездочнных отзывов

Собрать данные с помощью `requests` и `fake_useragent` не удалось, так как сайт обноруживал подозрительную активность. Ниже представлена функция, не реализованная из-за блокировки

```
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent

session = requests.session()
ua = UserAgent(verify_ssl=False)
```

```
def get_review(url, n):
    goods = set()
    bads = set()
    g_counter = 0
    b_counter = 0
    req = session.get(url, headers={'User-Agent': ua.random})
    soup = BeautifulSoup(req.text, 'html.parser')
    
    reviews = soup.find_all('a', {'class': 'review-title'})
    print(soup.title)
    for i in reviews:
        rev_link = i.get('href')
        rev_req = session.get(f'https://otzovik.com/{rev_link}', headers={'User-Agent': ua.random})
        rev_soup = BeautifulSoup(req.text, 'html.parser')
        # проверка на количество звезд
        stars = len(soup.find_all('div', {'class': 'product-rating tooltip-right'})[0].find_all('span', {'class': 'icons icon-star-1'}))
        print(stars)
        if stars == 5 and g_counter < n:
            rev_text = rev_soup.find_all('div', {'class': 'review-body description'})
            goods.add(rev_text[0].text)
            g_counter += 1
        elif stars == 1 and b_counter < n:
            rev_text = rev_soup.find_all('div', {'class': 'review-body description'})
            bads.add(rev_text[0].text)
            b_counter += 1
        
        if b_counter == n and g_counter == n: return (goods, bads)
        return (goods, bads)
```

### Selenium
Я переписала функцию, используя вместо библиотеки `requests` веб-драйвер `Selenium`

In [4]:
driver = webdriver.Chrome()

In [5]:
def get_reviews(url, n, page = 1): 
    reviews = {'good': set(), 'bad': set()}
    url_good = url + str(page) + '/?ratio=5'
    url_bad = url + str(page) + '?ratio=1'
    
    while len(reviews['good']) != n:
        driver.get(url_good)
        for i in range(0, 20):
            button = driver.find_elements(By.CLASS_NAME, 'review-title') 
            ActionChains(driver).click(button[i]).perform() # открываем конкретный отзыв
            try:
                rev_text = driver.find_element(By.XPATH, "//div[@class='review-body description']").text
                reviews['good'].add(rev_text)
                driver.get(url_good)
            except: driver.get(url_good)
            #можно выйти из цикла раньше
            if len(reviews['good']) == n: 
                break
        # листаем, если не хватило страниц
        page += 1
        url_good = url + str(page) + '/?ratio=5'
        
    while len(reviews['bad']) != n:
        driver.get(url_bad)
        for i in range(0, 20):
            button = driver.find_elements(By.CLASS_NAME, 'review-title') 
            ActionChains(driver).click(button[i]).perform() # открываем конкретный отзыв
            try:
                rev_text = driver.find_element(By.XPATH, "//div[@class='review-body description']").text
                reviews['bad'].add(rev_text)
                driver.get(url_bad)
            except: driver.get(url_bad)
            #можно выйти из цикла раньше
            if len(reviews['bad']) == n: 
                break
        # листаем, если не хватило страниц
        url_bad = url + str(page) + '?ratio=1'
        
    return reviews

In [6]:
reviews_text = {'good': set(), 'bad': set()}

In [12]:
driver = webdriver.Chrome()

In [8]:
for cin in tqdm(cinemas):
    new_rev = get_reviews(cin, 7)
    reviews_text['good'] = reviews_text['good'].union(new_rev['good'])
    reviews_text['bad'] = reviews_text['bad'].union(new_rev['bad'])

100%|██████████| 6/6 [04:32<00:00, 45.38s/it]


In [9]:
print(len(reviews_text['good']))
print(len(reviews_text['bad']))

42
42


## 2-3. Токенизация и составление частотного словря
*Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)*

*Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (2 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)*

In [10]:
# функция, которая токенизирует слова, приводит к нижнему регистру, начальной форме и сразу считает
def preprocess_and_count(texts):
    lemmas = Counter()
    for text in texts:
        for word in word_tokenize(text):
            if word.isalpha():
                word = morph.parse(word.lower())[0]
                lemma = word.normal_form
                lemmas.update(lemma.split())
    return lemmas

In [11]:
good_lemmas = preprocess_and_count(reviews_text['good'])

In [12]:
# посмотрим, как выглядят
for word, cont in zip(good_lemmas, range(0,10)):
    print(word)

ничего
не
мочь
сказать
о
это
кинотеатр
плохой
поскольку
я


In [13]:
bad_lemmas = preprocess_and_count(reviews_text['bad'])

In [14]:
# посмотрим, как выглядят
for word, cont in zip(bad_lemmas, range(0,10)):
    print(word)

являться
активный
пользователь
яндекс
плюс
и
быть
у
они
акция


In [15]:
print(len(good_lemmas))
print(len(bad_lemmas))

1712
1681


In [16]:
# столько должно убраться из каждого словаря
print(len(set(bad_lemmas).intersection(set(good_lemmas))))

756


In [17]:
# убираем повторения
good_lemmas_filtered = {}

bad_lemmas = dict(bad_lemmas)
good_lemmas = dict(good_lemmas)
for key, value in good_lemmas.items():
    if key in bad_lemmas:
        del bad_lemmas[key]
    else:
        good_lemmas_filtered[key] = value
        
bad_lemmas_filtered = bad_lemmas

In [18]:
for word, cont in zip(good_lemmas_filtered, range(0,10)):
    print(word)

плохой
некотрый
куртизанка
взрослый
мультсериал
великий
север
эксклюзив
безпринципный
легко


In [19]:
for word, cont in zip(bad_lemmas_filtered, range(0,10)):
    print(word)

активный
подстава
условие
означать
согласие
будущее
выпрыгивать
медиатека
отказаться
ссылка


In [20]:
print(len(good_lemmas_filtered))
print(len(bad_lemmas_filtered))

956
925


In [21]:
# фильтр по частотности
def get_freq_list(x, good_dict, bad_dict):
    freq_list = {'good': {}, 'bad': {}}
    for word, score in good_dict.items():
        if score > x:
            freq_list['good'][word] = score
    for word, score in bad_dict.items():
        if score > x:
            freq_list['bad'][word] = score
    return freq_list

In [22]:
freq_list = get_freq_list(1, good_lemmas_filtered, bad_lemmas_filtered)

In [23]:
print(len(freq_list['good']))
print(len(freq_list['bad']))

239
205


## 4. Определение тональности
*Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)*

In [24]:
def detect(freq_lists, text):
    counts = Counter({'good': 0, 'bad': 0})
    for word in text.split():
        if word in freq_lists['good']:
            counts['good'] += int(freq_lists['good'][word]) # добавляем сколько раз встречалось в отзвах этой тональности
        elif word in freq_lists['bad']:
            counts['bad'] += int(freq_lists['bad'][word])
    return counts.most_common(1)[0][0]

In [26]:
# соберем тестовый сэмпл
test_reviews = get_reviews(cinemas_test, 10)

In [27]:
print(len(test_reviews['good']))
print(len(test_reviews['bad']))

10
10


In [28]:
# функция, которая только токенизирует слова, приводит к нижнему регистру, начальной форме
def preprocess(texts):
    lemmas = set()
    for text in texts:
        lemmas_txt = ''
        for word in word_tokenize(text):
            if word.isalpha():
                word = morph.parse(word.lower())[0]
                lemma = word.normal_form
                lemmas_txt += lemma + " "
        lemmas.add(lemmas_txt)
    return lemmas

In [29]:
bad_lemmas_test = preprocess(test_reviews['bad'])
good_lemmas_test = preprocess(test_reviews['good'])

In [33]:
def test_detect(good_test, bad_test, freq_lists):
    results = []  # сюда будем писать результаты
    gold = []     # сюда будем писать исходный язык
    for review in good_test:
        predicted_mood = detect(freq_lists, review)
        results.append(predicted_mood)
        gold.append('good')
    for review in bad_test:
        predicted_mood = detect(freq_lists, review)
        results.append(predicted_mood)
        gold.append('bad')
    print("Accuracy:", accuracy_score(results, gold))

In [34]:
test_detect(good_lemmas_test, bad_lemmas_test, freq_list)

Accuracy: 0.7


## 5. Улучшения
*Предложите как минимум 2 способа улучшить этот алгоритм определения тональности отзыва (1 балл за описание и реализацию каждого способа; если 2 способа описаны только текстом, это 1 балл. За третий и последующие способы дополнительных баллов не будет)*

1. при определении тональности учитывать не просто вхождение слова, но и его "вес", то есть частотность в отзывах той или иной тональности (уже есть в коде)
2. брать не только содержание, но и "достоинства", "недостатки" (если брать только эти разделы, то будет очень мало материала, поэтому можно объединить с исходным). Как правило, даже в положительных отзывах указыают недостатки, поэтому преимущество такого метода в том, что можно не фильтровать отзывы по количеству звезд, а брать все подряд

In [36]:
def get_pros_n_cons(url, n, page = 1): 
    reviews = {'pros': set(), 'cons': set()}
    url = url + str(page) + '/'
    driver.get(url)
    
    for i in range(0, n):
        button = driver.find_elements(By.CLASS_NAME, 'review-title') 
        ActionChains(driver).click(button[i]).perform() # открываем конкретный отзыв
        pros = driver.find_element(By.XPATH, "//div[@class='review-plus']").text
        reviews['pros'].add(pros)
        cons = driver.find_element(By.XPATH, "//div[@class='review-minus']").text
        reviews['cons'].add(cons)
        driver.get(url)
    
    return reviews

In [55]:
pros_n_cons = {'pros': set(), 'cons': set()}

In [53]:
driver = webdriver.Chrome()

In [56]:
for cin in tqdm(cinemas):
    new_pc = get_pros_n_cons(cin, 20)
    pros_n_cons['pros'] = pros_n_cons['pros'].union(new_pc['pros'])
    pros_n_cons['cons'] = pros_n_cons['cons'].union(new_pc['cons'])

100%|██████████| 6/6 [05:53<00:00, 58.92s/it]


In [57]:
for word, cont in zip(pros_n_cons['pros'], range(0,10)):
    print(word)

Хороший выбор фильмов, не виснет, всегда доступен.
Удобно использовать в любом месте
Не хуже прочих, оригинальный контент от Netflix
Есть неплохие фильмы
Их очень мало, а может это и вовсе не плюсы
Просмотр доступен на любом устройстве. Большой выбор от сериалов до фильмов
✅Хорошее качество изображения. ✅Возможность одновременно смотреть на 6 устройствах. ✅Функция управления просмотром (рекламу долой). ✅Возможность аренды оборудования.
Фильмы и сериалы без рекламы
Много фильмов, которых нет в свободном доступе
На фоне конкурентов их нет


In [58]:
for word, cont in zip(pros_n_cons['cons'], range(0,10)):
    print(word)

Есть платная подписка
Делаем выводы
списание средств за продление при фактически отключенной подписке
Дерьмо
платный контент с завышенной ценой, но кто им пользуется?!
Одни минусы
Меняют озвучку с хорошей, на ужасную
Привязка к карте, часть фильмов отдельно платные
Нету
не активируют промо-коды


In [59]:
pros_lemmas = preprocess_and_count(pros_n_cons['pros'])

In [60]:
cons_lemmas = preprocess_and_count(pros_n_cons['cons'])

In [61]:
# убираем повторения
pros_filtered = {}

cons_lemmas = dict(cons_lemmas)
pros_lemmas = dict(pros_lemmas)
for key, value in pros_lemmas.items():
    if key in cons_lemmas:
        del cons_lemmas[key]
    else:
        pros_filtered[key] = value
        
cons_filtered = cons_lemmas

In [62]:
pros_cons_freq = get_freq_list(1, pros_lemmas, cons_lemmas)

In [63]:
# проверяем точность с просто плюсами и минусами
test_detect(good_lemmas_test, bad_lemmas_test, pros_cons_freq)

Accuracy: 0.5


Вообще не работает. Может, этот список поможет улучшить изначальные данные

In [64]:
pros_cons_freq.update(freq_list)

In [65]:
# проверяем точность на всем
test_detect(good_lemmas_test, bad_lemmas_test, pros_cons_freq)

Accuracy: 0.7


Возможно, результат станет лучше, если взять больше плюсов и минусов. В любом случае, это один из способов добыть больше материала той или иной тональности